In [107]:
import os
import glob
from pathlib import Path
import yaml

import logging

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# Show more columns
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 200)

# Initiate Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [108]:
def get_directories():

    # Get Current Working Directory
    CWD = Path().resolve()

    # 
    # Priority: ENV → script → Jupyter fallback
    env_root = os.getenv("APP_ROOT")

    if env_root:
        APP_ROOT = Path(env_root).resolve()

    else:
        try:
            # Script Path
            APP_ROOT = Path(__file__).resolve().parents[1]
        except NameError:
            # Notebook execution path
            APP_ROOT = CWD.parent

    # Set Data Directory
    DATA_DIR = APP_ROOT / "data"

    # Log everything for debugging
    logger.info(f"CWD:             {CWD}")
    logger.info(f"APP_ROOT:        {APP_ROOT}")
    logger.info(f"DATA Directory:  {DATA_DIR}")

    return APP_ROOT, DATA_DIR

#### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 

def load_data(DATA_PATH, DATA_FILE, **kwargs):

    """
        Load data file with
    """

    try:
        data = pd.read_csv(DATA_PATH/DATA_FILE, **kwargs)
        logger.info(f"Data File Loaded: {data.info()}")
        return data

    except Exception as e:
        print(f"Error reading CSV file: {e}")
        return None

    
    #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### #### 


In [109]:
# Global Variables

# Folder Locations
APP_ROOT, DATA_DIR = get_directories()

RAW_DATA_DIR = DATA_DIR / "raw"
RAW_DATA_SUBDIR = "NASA_Turbofan_Jet_Engine_Dataset/CMaps"

RAW_DATA_PATH = RAW_DATA_DIR / RAW_DATA_SUBDIR

BRONZE_DATA_DIR = DATA_DIR / "bronze"
BRONZE_DATA_FILE_NAME = "turbofan_test_bronze.csv"
BRONZE_DATA_OUTPUT_PATH = BRONZE_DATA_DIR / BRONZE_DATA_FILE_NAME


INFO:__main__:CWD:             /notebooks
INFO:__main__:APP_ROOT:        /
INFO:__main__:DATA Directory:  /data


In [110]:
print(RAW_DATA_PATH)

/data/raw/NASA_Turbofan_Jet_Engine_Dataset/CMaps


In [111]:
# There are 26 columns in the dataset. 
# The first five columns have a different format then the remaining 21. 
# We will addresss these columns first then add in the remainingg 21. 

# Creating column name list for the first five. 
COLUMN_NAMES = [
    "unit",
    "cycle",
    "op_setting_1",
    "op_setting_2",
    "op_setting_3",
]

# As stated the remaining 21 columns share a similar name structure of sensor then a number
# So we will generate them using a loop and appended them to our list:
COLUMN_NAMES += [f"s{i}" for i in range(1, 22)]

# Display Column Names to verify success.
len(COLUMN_NAMES), COLUMN_NAMES[:10]


(26,
 ['unit',
  'cycle',
  'op_setting_1',
  'op_setting_2',
  'op_setting_3',
  's1',
  's2',
  's3',
  's4',
  's5'])

In [112]:
data_import_parameters = {"sep":"\s+", "header":None, "names": COLUMN_NAMES}

rul_column_names = ['RUL_final']

rul_import_parameters = {"sep":"\s+", "header": None, "names": rul_column_names}

FD_IDS = ["FD001", "FD002", "FD003", "FD004"]

# Anomaly window for when errors occur
ANOMALY_HORIZON = 30

# Creating a list 
all_testing_dfs = []

for fd_id in FD_IDS:

    RAW_FD_PATH = RAW_DATA_PATH

    RAW_DATA_FILE = f"test_{fd_id}.txt"
    RAW_RUL_FILE = f"RUL_{fd_id}.txt"

    raw_df = load_data(RAW_FD_PATH, RAW_DATA_FILE, **data_import_parameters)
    logger.info(f"Importing Test File: {raw_df.info()}")

    raw_df = raw_df.reset_index(drop=True)
    raw_df.index.name = None


    max_cycle = raw_df.groupby("unit")["cycle"].max().rename("last_cycle")


    raw_df = raw_df.merge(max_cycle, on="unit" , how="left")
    raw_df.info()
    raw_df.head()


    rul_df = load_data(RAW_FD_PATH, RAW_RUL_FILE, **rul_import_parameters)
    rul_df.columns = ['RUL_final']

    rul_df["unit"] = range(1, len(rul_df)+1)

    df = raw_df.merge(rul_df, on="unit", how="left")
    df['failure_cycle'] = df['last_cycle'] + df['RUL_final']
    df['RUL'] = df['failure_cycle'] - df['cycle']

    df = df.drop(columns=['last_cycle', 'failure_cycle'])

    df['anomaly_flag'] = (df["RUL"] <= ANOMALY_HORIZON).astype(int)
    logger.info(f"Anomaly Flag Added: {df.info()}")

    df["dataset_name"] = "TURBOFAN"
    logger.info(f"Column Added: 'dataset_name': {df.info()}")

    df["fd_id"] = fd_id
    logger.info(f"Column Added: 'fd_id': {df.info()}")

    df["event_time"] = df["cycle"]
    logger.info(f"Column Added: 'event_time': {df.info()}")

    df = df.rename(columns={
                "unit": "machine_id",
                "cycle": "time_index",
                })
    logger.info(f"Columns Renamed: 'unit'>'machine_id' & 'cycle'>'time_index'  {df.info()}")

    core_columns = ["dataset_name", "fd_id", "machine_id", "time_index", "event_time"]
    logger.info(f"Core Columns Assigned and stored to list: {core_columns}")

    label_columns = ["RUL", "anomaly_flag"]
    logger.info(f"Label Columns Assigned and stored to list: {label_columns}")

    data_columns = list(df.columns)
    logger.info(f"Data Columns List created from existing columns in dataset: {data_columns}")

    for column in core_columns + label_columns: 
        if column in data_columns:
            data_columns.remove(column)

    logger.info(f"Data Columns List purged of core and label columns: {data_columns}")

    ordered_columns = core_columns + label_columns + data_columns
    logger.info(f"Column Order Set: {ordered_columns}")

    df_ordered = df[ordered_columns].copy()
    logger.info(f"Orderd Dataframe created: {df_ordered.info()}")

    all_testing_dfs.append(df_ordered)


bronze_dfs = pd.concat(all_testing_dfs, axis=0, ignore_index=True)
logger.info(f"All Dataframe concatenated into a single dataframe, vertically {bronze_dfs}")

INFO:__main__:Data File Loaded: None
INFO:__main__:Importing Test File: None


INFO:__main__:Data File Loaded: None
INFO:__main__:Anomaly Flag Added: None
INFO:__main__:Column Added: 'dataset_name': None
INFO:__main__:Column Added: 'fd_id': None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13096 entries, 0 to 13095
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   unit          13096 non-null  int64  
 1   cycle         13096 non-null  int64  
 2   op_setting_1  13096 non-null  float64
 3   op_setting_2  13096 non-null  float64
 4   op_setting_3  13096 non-null  float64
 5   s1            13096 non-null  float64
 6   s2            13096 non-null  float64
 7   s3            13096 non-null  float64
 8   s4            13096 non-null  float64
 9   s5            13096 non-null  float64
 10  s6            13096 non-null  float64
 11  s7            13096 non-null  float64
 12  s8            13096 non-null  float64
 13  s9            13096 non-null  float64
 14  s10           13096 non-null  float64
 15  s11           13096 non-null  float64
 16  s12           13096 non-null  float64
 17  s13           13096 non-null  float64
 18  s14           13096 non-nu

INFO:__main__:Column Added: 'event_time': None
INFO:__main__:Columns Renamed: 'unit'>'machine_id' & 'cycle'>'time_index'  None
INFO:__main__:Core Columns Assigned and stored to list: ['dataset_name', 'fd_id', 'machine_id', 'time_index', 'event_time']
INFO:__main__:Label Columns Assigned and stored to list: ['RUL', 'anomaly_flag']
INFO:__main__:Data Columns List created from existing columns in dataset: ['machine_id', 'time_index', 'op_setting_1', 'op_setting_2', 'op_setting_3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21', 'RUL_final', 'RUL', 'anomaly_flag', 'dataset_name', 'fd_id', 'event_time']
INFO:__main__:Data Columns List purged of core and label columns: ['op_setting_1', 'op_setting_2', 'op_setting_3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21', 'RUL_final']
INFO:__main__:Column Order Set: ['data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13096 entries, 0 to 13095
Data columns (total 32 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   unit          13096 non-null  int64  
 1   cycle         13096 non-null  int64  
 2   op_setting_1  13096 non-null  float64
 3   op_setting_2  13096 non-null  float64
 4   op_setting_3  13096 non-null  float64
 5   s1            13096 non-null  float64
 6   s2            13096 non-null  float64
 7   s3            13096 non-null  float64
 8   s4            13096 non-null  float64
 9   s5            13096 non-null  float64
 10  s6            13096 non-null  float64
 11  s7            13096 non-null  float64
 12  s8            13096 non-null  float64
 13  s9            13096 non-null  float64
 14  s10           13096 non-null  float64
 15  s11           13096 non-null  float64
 16  s12           13096 non-null  float64
 17  s13           13096 non-null  float64
 18  s14           13096 non-nu

...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33991 entries, 0 to 33990
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   unit          33991 non-null  int64  
 1   cycle         33991 non-null  int64  
 2   op_setting_1  33991 non-null  float64
 3   op_setting_2  33991 non-null  float64
 4   op_setting_3  33991 non-null  float64
 5   s1            33991 non-null  float64
 6   s2            33991 non-null  float64
 7   s3            33991 non-null  float64
 8   s4            33991 non-null  float64
 9   s5            33991 non-null  float64
 10  s6            33991 non-null  float64
 11  s7            33991 non-null  float64
 12  s8            33991 non-null  float64
 13  s9            33991 non-null  float64
 14  s10           33991 non-null  float64
 15  s11           33991 non-null  float64
 16  s12           33991 non-null  float64
 17  s13           33991 non-null  float64
 18  s14           33991 non-nu

INFO:__main__:Data File Loaded: None
INFO:__main__:Importing Test File: None
INFO:__main__:Data File Loaded: None
INFO:__main__:Anomaly Flag Added: None
INFO:__main__:Column Added: 'dataset_name': None
INFO:__main__:Column Added: 'fd_id': None
INFO:__main__:Column Added: 'event_time': None
INFO:__main__:Columns Renamed: 'unit'>'machine_id' & 'cycle'>'time_index'  None
INFO:__main__:Core Columns Assigned and stored to list: ['dataset_name', 'fd_id', 'machine_id', 'time_index', 'event_time']
INFO:__main__:Label Columns Assigned and stored to list: ['RUL', 'anomaly_flag']
INFO:__main__:Data Columns List created from existing columns in dataset: ['machine_id', 'time_index', 'op_setting_1', 'op_setting_2', 'op_setting_3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21', 'RUL_final', 'RUL', 'anomaly_flag', 'dataset_name', 'fd_id', 'event_time']
INFO:__main__:Data Columns List purged of core and label col

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16596 entries, 0 to 16595
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   unit          16596 non-null  int64  
 1   cycle         16596 non-null  int64  
 2   op_setting_1  16596 non-null  float64
 3   op_setting_2  16596 non-null  float64
 4   op_setting_3  16596 non-null  float64
 5   s1            16596 non-null  float64
 6   s2            16596 non-null  float64
 7   s3            16596 non-null  float64
 8   s4            16596 non-null  float64
 9   s5            16596 non-null  float64
 10  s6            16596 non-null  float64
 11  s7            16596 non-null  float64
 12  s8            16596 non-null  float64
 13  s9            16596 non-null  float64
 14  s10           16596 non-null  float64
 15  s11           16596 non-null  float64
 16  s12           16596 non-null  float64
 17  s13           16596 non-null  float64
 18  s14           16596 non-nu

INFO:__main__:Data File Loaded: None
INFO:__main__:Importing Test File: None
INFO:__main__:Data File Loaded: None
INFO:__main__:Anomaly Flag Added: None
INFO:__main__:Column Added: 'dataset_name': None
INFO:__main__:Column Added: 'fd_id': None
INFO:__main__:Column Added: 'event_time': None
INFO:__main__:Columns Renamed: 'unit'>'machine_id' & 'cycle'>'time_index'  None
INFO:__main__:Core Columns Assigned and stored to list: ['dataset_name', 'fd_id', 'machine_id', 'time_index', 'event_time']
INFO:__main__:Label Columns Assigned and stored to list: ['RUL', 'anomaly_flag']
INFO:__main__:Data Columns List created from existing columns in dataset: ['machine_id', 'time_index', 'op_setting_1', 'op_setting_2', 'op_setting_3', 's1', 's2', 's3', 's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21', 'RUL_final', 'RUL', 'anomaly_flag', 'dataset_name', 'fd_id', 'event_time']
INFO:__main__:Data Columns List purged of core and label col

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41214 entries, 0 to 41213
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   unit          41214 non-null  int64  
 1   cycle         41214 non-null  int64  
 2   op_setting_1  41214 non-null  float64
 3   op_setting_2  41214 non-null  float64
 4   op_setting_3  41214 non-null  float64
 5   s1            41214 non-null  float64
 6   s2            41214 non-null  float64
 7   s3            41214 non-null  float64
 8   s4            41214 non-null  float64
 9   s5            41214 non-null  float64
 10  s6            41214 non-null  float64
 11  s7            41214 non-null  float64
 12  s8            41214 non-null  float64
 13  s9            41214 non-null  float64
 14  s10           41214 non-null  float64
 15  s11           41214 non-null  float64
 16  s12           41214 non-null  float64
 17  s13           41214 non-null  float64
 18  s14           41214 non-nu

In [113]:
df.head(30)

,machine_id,time_index,op_setting_1,op_setting_2,op_setting_3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL_final,RUL,anomaly_flag,dataset_name,fd_id,event_time
0,1,1,20.0072,0.7000,100.0,491.19,606.67,1481.04,1227.81,9.35,13.60,332.52,2323.67,8704.98,1.07,43.83,313.03,2387.78,8048.98,9.2229,0.02,362,2324,100.00,24.31,14.7007,22,251,0,TURBOFAN,FD004,1
1,1,2,24.9984,0.6200,60.0,462.54,536.22,1256.17,1031.48,7.05,9.00,174.46,1915.21,7999.94,0.93,36.11,163.61,2028.09,7863.46,10.8632,0.02,306,1915,84.93,14.36,8.5748,22,250,0,TURBOFAN,FD004,2
2,1,3,42.0000,0.8420,100.0,445.00,549.23,1340.13,1105.88,3.91,5.69,137.34,2211.93,8305.38,1.01,41.52,129.98,2387.95,8071.13,9.3960,0.02,328,2212,100.00,10.39,6.4365,22,249,0,TURBOFAN,FD004,3
3,1,4,42.0035,0.8402,100.0,445.00,549.19,1339.70,1107.26,3.91,5.69,137.23,2211.89,8301.00,1.01,41.73,129.48,2387.90,8078.89,9.3594,0.02,328,2212,100.00,10.56,6.2367,22,248,0,TURBOFAN,FD004,4
4,1,5,35.0079,0.8400,100.0,449.44,555.10,1353.04,1117.80,5.48,7.97,192.94,2222.71,8331.05,1.02,41.32,181.82,2387.87,8057.83,9.3030,0.02,333,2223,100.00,14.85,8.9326,22,247,0,TURBOFAN,FD004,5
5,1,6,25.0010,0.6203,60.0,462.54,536.40,1255.38,1043.74,7.05,9.00,174.32,1915.27,8001.14,0.93,36.41,163.95,2028.12,7867.38,10.8661,0.02,304,1915,84.93,14.17,8.6135,22,246,0,TURBOFAN,FD004,6
6,1,7,19.9996,0.7015,100.0,491.19,607.29,1473.03,1230.62,9.35,13.60,332.63,2323.64,8708.79,1.07,43.94,312.77,2387.83,8052.97,9.1925,0.02,362,2324,100.00,24.52,14.6965,22,245,0,TURBOFAN,FD004,7
7,1,8,35.0015,0.8414,100.0,449.44,554.81,1361.07,1115.14,5.48,7.97,193.45,2222.76,8328.97,1.02,41.39,182.23,2387.84,8055.81,9.3004,0.02,331,2223,100.00,14.84,8.9004,22,244,0,TURBOFAN,FD004,8
8,1,9,19.9980,0.7000,100.0,491.19,607.35,1481.89,1240.15,9.35,13.60,332.27,2323.66,8705.64,1.07,43.79,312.20,2387.76,8052.46,9.1965,0.02,362,2324,100.00,24.34,14.6079,22,243,0,TURBOFAN,FD004,9
9,1,10,42.0033,0.8416,100.0,445.00,549.30,1342.24,1106.32,3.91,5.69,136.65,2211.85,8301.07,1.01,41.65,129.69,2387.97,8075.89,9.3860,0.02,328,2212,100.00,10.45,6.3830,22,242,0,TURBOFAN,FD004,10


In [114]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41214 entries, 0 to 41213
Data columns (total 32 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   machine_id    41214 non-null  int64  
 1   time_index    41214 non-null  int64  
 2   op_setting_1  41214 non-null  float64
 3   op_setting_2  41214 non-null  float64
 4   op_setting_3  41214 non-null  float64
 5   s1            41214 non-null  float64
 6   s2            41214 non-null  float64
 7   s3            41214 non-null  float64
 8   s4            41214 non-null  float64
 9   s5            41214 non-null  float64
 10  s6            41214 non-null  float64
 11  s7            41214 non-null  float64
 12  s8            41214 non-null  float64
 13  s9            41214 non-null  float64
 14  s10           41214 non-null  float64
 15  s11           41214 non-null  float64
 16  s12           41214 non-null  float64
 17  s13           41214 non-null  float64
 18  s14           41214 non-nu

In [115]:
bronze_dfs.shape

(104897, 32)

In [116]:
bronze_dfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104897 entries, 0 to 104896
Data columns (total 32 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   dataset_name  104897 non-null  object 
 1   fd_id         104897 non-null  object 
 2   machine_id    104897 non-null  int64  
 3   time_index    104897 non-null  int64  
 4   event_time    104897 non-null  int64  
 5   RUL           104897 non-null  int64  
 6   anomaly_flag  104897 non-null  int64  
 7   op_setting_1  104897 non-null  float64
 8   op_setting_2  104897 non-null  float64
 9   op_setting_3  104897 non-null  float64
 10  s1            104897 non-null  float64
 11  s2            104897 non-null  float64
 12  s3            104897 non-null  float64
 13  s4            104897 non-null  float64
 14  s5            104897 non-null  float64
 15  s6            104897 non-null  float64
 16  s7            104897 non-null  float64
 17  s8            104897 non-null  float64
 18  s9  

In [117]:
bronze_dfs.head(10)

,dataset_name,fd_id,machine_id,time_index,event_time,RUL,anomaly_flag,op_setting_1,op_setting_2,op_setting_3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,RUL_final
0,TURBOFAN,FD001,1,1,1,142,0,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,21.61,553.90,2388.04,9050.17,1.3,47.20,521.72,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,112
1,TURBOFAN,FD001,1,2,2,141,0,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,21.61,554.85,2388.01,9054.42,1.3,47.50,522.16,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,112
2,TURBOFAN,FD001,1,3,3,140,0,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,21.61,554.11,2388.05,9056.96,1.3,47.50,521.97,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,112
3,TURBOFAN,FD001,1,4,4,139,0,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,21.61,554.07,2388.03,9045.29,1.3,47.28,521.38,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,112
4,TURBOFAN,FD001,1,5,5,138,0,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,21.61,554.16,2388.01,9044.55,1.3,47.31,522.15,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,112
5,TURBOFAN,FD001,1,6,6,137,0,0.0012,0.0003,100.0,518.67,642.11,1579.12,1395.13,14.62,21.61,554.22,2388.00,9050.96,1.3,47.26,521.92,2388.08,8127.46,8.4238,0.03,392,2388,100.0,38.91,23.3467,112
6,TURBOFAN,FD001,1,7,7,136,0,-0.0000,0.0002,100.0,518.67,642.11,1583.34,1404.84,14.62,21.61,553.89,2388.05,9051.39,1.3,47.31,522.01,2388.06,8134.97,8.3914,0.03,391,2388,100.0,38.85,23.3952,112
7,TURBOFAN,FD001,1,8,8,135,0,0.0006,-0.0000,100.0,518.67,642.54,1580.89,1400.89,14.62,21.61,553.59,2388.05,9052.86,1.3,47.21,522.09,2388.06,8125.93,8.4213,0.03,393,2388,100.0,39.05,23.3224,112
8,TURBOFAN,FD001,1,9,9,134,0,-0.0036,0.0000,100.0,518.67,641.88,1593.29,1412.28,14.62,21.61,554.49,2388.06,9048.55,1.3,47.37,522.03,2388.05,8134.15,8.4353,0.03,391,2388,100.0,39.10,23.4521,112
9,TURBOFAN,FD001,1,10,10,133,0,-0.0025,-0.0001,100.0,518.67,642.07,1585.25,1398.64,14.62,21.61,554.28,2388.04,9051.95,1.3,47.14,522.00,2388.06,8134.08,8.4093,0.03,391,2388,100.0,38.87,23.3820,112


In [118]:
bronze_dfs.to_csv(BRONZE_DATA_OUTPUT_PATH, index=False)